# Bini Fast Matrix Multiplication O(2.7799)

In [1]:
import numpy as np

In [2]:
from numpy import linalg as la

In [3]:
np.set_printoptions(precision=2)

In [ ]:
#without dynamic peeling:
# row of A need to be divisible by 3 (power of is preferable)
# columns of A need do be divisble by 2

# rows and cols of B need to be divisible by two

In [4]:
A = np.random.rand(81, 32)
B = np.random.rand(32,32)

In [ ]:
def bini(A, B, steps, e=0.1):
    
    #Check Dimensions
    (m, n) = A.shape
    (nn, p) = B.shape
    if n != nn: raise ValueError("incompatible dimensions")
    
    #pre-allocate output matrix
    C = np.zeros((m,p))
    
    #Base case
    if steps == 0 or m == 1 or n == 1 or p == 1:
        C = np.dot(A,B)
        return C
    
    #Dynamic peling
    #**************
    # right now only input matricies that fit the power of 3 and 2 dimension rules
    
    # split up the matricies once rows of A are divisible by 3
    # and cols of A and rows and cols of are divisible by 2
    